In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str
    preprocessor_path: Path

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

[2025-02-21 23:30:17,662 : INFO : __init__ : Logger has been set up successfully!]


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
        root_dir=config.root_dir,
        data_path=config.data_path,
        target_column=config.target_column,
        preprocessor_path=config.preprocessor_path
        )
    
        return data_transformation_config

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
from mlproject import logger

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.columns_to_drop = ['Unnamed: 0', 'flight', 'duration']
        self.target_column = 'price'

    def train_test_spliting(self):
        try:
            data = pd.read_csv(self.config.data_path)
            
            # Drop specified columns
            data.drop(self.columns_to_drop, axis=1, inplace=True)
            logger.info(f"Dropped columns: {self.columns_to_drop}")
            
            train, test = train_test_split(data, test_size=0.25, random_state=42)

            train_path = os.path.join(self.config.root_dir, "train.csv")
            test_path = os.path.join(self.config.root_dir, "test.csv")
            train.to_csv(train_path, index=False)
            test.to_csv(test_path, index=False)

            logger.info("Split data into training and test sets")
            logger.info(f"Training data shape: {train.shape}")
            logger.info(f"Test data shape: {test.shape}")

            return train, test
            
        except Exception as e:
            logger.error(f"Error in train_test_splitting: {e}")
            raise e
    
    def preprocess_features(self, train, test):
        try:
            # Identify numerical and categorical columns
            numerical_columns = train.select_dtypes(include=["int64", "float64"]).columns
            categorical_columns = train.select_dtypes(include=["object", "category"]).columns

            # Exclude the target column from numerical columns
            if self.target_column in numerical_columns:
                numerical_columns = numerical_columns.drop(self.target_column)

            logger.info(f"Numerical columns: {list(numerical_columns)}")
            logger.info(f"Categorical columns: {list(categorical_columns)}")

            # Preprocessing pipelines
            num_pipeline = Pipeline(steps=[
                ("scaler", StandardScaler())
            ])
            cat_pipeline = Pipeline(steps=[
                ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
            ])
            
            preprocessor = ColumnTransformer(
                transformers=[
                    ("num", num_pipeline, numerical_columns),
                    ("cat", cat_pipeline, categorical_columns)
                ],
                remainder="passthrough"
            )

            # Separate features and target
            train_x = train.drop(columns=[self.target_column])
            test_x = test.drop(columns=[self.target_column])
            train_y = train[self.target_column]
            test_y = test[self.target_column]

            # Fit preprocessor and transform features
            train_processed = preprocessor.fit_transform(train_x)
            test_processed = preprocessor.transform(test_x)

            # Ensure target is 2D array
            train_y = train_y.values.reshape(-1, 1)
            test_y = test_y.values.reshape(-1, 1)

            # Combine processed features with target
            train_combined = np.hstack((train_processed, train_y))
            test_combined = np.hstack((test_processed, test_y))

            # Save preprocessor
            joblib.dump(preprocessor, self.config.preprocessor_path)
            logger.info(f"Preprocessor saved at {self.config.preprocessor_path}")

            # Save processed data
            np.save(os.path.join(self.config.root_dir, "train_processed.npy"), train_combined)
            np.save(os.path.join(self.config.root_dir, "test_processed.npy"), test_combined)

            logger.info("Preprocessed train and test data saved successfully.")
            return train_processed, test_processed

        except Exception as e:
            logger.error(f"Error in preprocess_features: {e}")
            raise e

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    train,test = data_transformation.train_test_spliting()
    train_processed, test_processed = data_transformation.preprocess_features(train, test)

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-02-21 23:30:20,148 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-02-21 23:30:20,154 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-02-21 23:30:20,159 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-02-21 23:30:20,162 : INFO : common : created directory at: artifacts]
[2025-02-21 23:30:20,164 : INFO : common : created directory at: artifacts/data_transformation]


Error in preprocess_data: 'TotalCharges'
Error in train_test_spliting: 'TotalCharges'
Missing key in configuration: 'TotalCharges'
